检查valid函数的正确性

In [ ]:
from rlepose.trainer import train, validate, validate_gt, validate_gt_3d

from rlepose.utils.transforms import get_coord

import rlepose.datasets.lxd_freihand as lxd_freihand
from rlepose.models import builder
from rlepose.utils.config import update_config

import torch
cfg = update_config("./configs/256x192_res50_regress-flow_freihand.yaml")

def preset_model(cfg):
    model = builder.build_sppe(cfg.MODEL, preset_cfg=cfg.DATA_PRESET)

    if cfg.MODEL.PRETRAINED:
        model.load_state_dict(torch.load(cfg.MODEL.PRETRAINED))
    elif cfg.MODEL.TRY_LOAD:
        pretrained_state = torch.load(cfg.MODEL.TRY_LOAD)
        model_state = model.state_dict()
        pretrained_state = {k: v for k, v in pretrained_state.items()
                            if k in model_state and v.size() == model_state[k].size()}

        model_state.update(pretrained_state)
        model.load_state_dict(model_state)
    else:
        model._initialize()

    return model

m = preset_model(cfg)
m.train()

hm_shape = cfg.DATA_PRESET.get('HEATMAP_SIZE')
depth_dim = cfg.MODEL.get('DEPTH_DIM')
output_3d = cfg.DATA_PRESET.get('OUT_3D', False)
hm_shape = (hm_shape[1], hm_shape[0], depth_dim)
grad_clip = cfg.TRAIN.get('GRAD_CLIP', False)



heatmap_to_coord = get_coord(cfg, cfg.DATA_PRESET.HEATMAP_SIZE, output_3d)

gt_val_dataset = builder.build_dataset(cfg.DATASET.VAL, preset_cfg=cfg.DATA_PRESET, train=False, heatmap2coord=cfg.TEST.HEATMAP2COORD)
gt_val_sampler = torch.utils.data.distributed.DistributedSampler(
    gt_val_dataset, num_replicas=cfg.TRAIN.WORLD_SIZE, rank=1)

gt_val_loader = torch.utils.data.DataLoader(
    gt_val_dataset, batch_size=32, shuffle=False, num_workers=20, drop_last=False, sampler=gt_val_sampler)


# gt_AP = validate_gt(m, opt, cfg, heatmap_to_coord)


In [ ]:
for inps, labels in gt_val_loader:
    bboxes = labels.pop('bbox')
    raise

In [ ]:
bboxes.shape

In [ ]:
import cv2
import numpy as np
import math
import torch
import torch
from rlepose.models import builder
from rlepose.utils.valid_utils_lxd import cal_mAP_RMSE
import time
cfg = update_config("./configs/256x192_res50_regress-flow_freihand.yaml")

def paint(im, kpts):
    '''
    输入为单帧的im与其对应的一组kpts
    '''
    
    colors = [[255, 0, 0], [255, 85, 0], [255, 170, 0], [255, 255, 0], [170, 255, 0], [85, 255, 0], [0, 255, 0], \
          [0, 255, 85], [0, 255, 170], [0, 255, 255], [0, 170, 255], [0, 85, 255], [0, 0, 255], \
          [128, 0, 128], [255, 192, 203], [0, 128, 128], [128, 128, 0], [128, 0, 0], [0, 128, 0], [0, 0, 128]]
    limbSeq = [[0,1],[1,2],[2,3],[3,4],[0,5],[5,6],[6,7],[7,8],[0,9],[9,10],[10,11],[11,12],[0,13],[13,14],[14,15],[15,16],[0,17],[17,18],[18,19],[19,20]]

    # im = cv2.imread(img_path)
    # draw points
    for k in kpts:
        x = int(k[0])
        y = int(k[1])
        # print(f'im .shape = {im.shape}')
        # print(f'x={x}, y={y}')
        cv2.circle(im, (x, y), 2, (0, 0, 255), -1)

    # draw lines
    # print("kpts shape: ", kpts)
    for i in range(len(limbSeq)):
        cur_im = im.copy()
        limb = limbSeq[i]
        # print(kpts[limb[0]])
        [Y0, X0] = kpts[limb[0]]
        [Y1, X1] = kpts[limb[1]]
        mX = np.mean([X0, X1])
        mY = np.mean([Y0, Y1])
        length = ((X0 - X1) ** 2 + (Y0 - Y1) ** 2) ** 0.5
        angle = math.degrees(math.atan2(X0 - X1, Y0 - Y1))
        polygon = cv2.ellipse2Poly((int(mY), int(mX)), (int(length / 2), 4), int(angle), 0, 360, 1)
        cv2.fillConvexPoly(cur_im, polygon, colors[i])
        im = cv2.addWeighted(im, 0.4, cur_im, 0.6, 0)

    return im




device = torch.device('cuda')
m = builder.build_sppe(cfg.MODEL, preset_cfg=cfg.DATA_PRESET)  # 根据cfg的配置信息构建模型

m.load_state_dict(torch.load("/home/louxd/ldlib/res-loglikelihood-regression-master/weights/model_114.pth", map_location='cpu'), 
                  strict=True)  # 加载权重

m.cuda(device)  # 把模型放到gpu中

time_cost_list = []
with torch.no_grad():
    gt_val_dataset = builder.build_dataset(cfg.DATASET.VAL, preset_cfg=cfg.DATA_PRESET, train=False, heatmap2coord=cfg.TEST.HEATMAP2COORD)
    gt_val_sampler = torch.utils.data.distributed.DistributedSampler(
        gt_val_dataset, num_replicas=1, rank=0)

    gt_val_loader = torch.utils.data.DataLoader(
        gt_val_dataset, batch_size=16, shuffle=False, num_workers=20, drop_last=False, sampler=gt_val_sampler)
    
    
    mAP_ary = []
    RMSE_ary = []
    for index, (inps, labels) in enumerate(gt_val_loader):
        inps = inps.cuda(device)

        for k, _ in labels.items():
            if k == 'type':
                continue
            
            labels[k] = labels[k].cuda(device)

        reason_start_time = time.time()  # 记录开始时间
        output = m(inps, labels)
        reaon_end_time = time.time()  # 记录结束时间
        reason_cost_time = reaon_end_time - reason_start_time  # 计算耗时，单位为秒
        time_cost_list.append(reason_cost_time)

        if isinstance(inps, list):
            batch_size = inps[0].size(0)
        else:
            batch_size = inps.size(0)

        kpts_gt = labels['target_uv'].cpu().numpy().reshape(-1, 21, 2) # size * 42
        kpts_pre = output.pred_jts.cpu().numpy().reshape(-1, 21, 2)

        mAP_str, RMSE = cal_mAP_RMSE(kpts_pre, kpts_gt)
        
        # print & draw
        for i in range(len(inps)):
            if i == 0 & index < 100:
                #print(f'batch:{batch_idx}_{i}')
                imgi = inps[i].cpu().numpy()
                imgi = np.transpose(imgi, (1, 2, 0))
                imgi = (imgi + np.array([0.480, 0.457, 0.406], dtype=np.float32)) * np.array([255., 255., 255.], dtype=np.float32)
                imgi = cv2.cvtColor(imgi, cv2.COLOR_BGR2RGB) 
                img_h, img_w, _ = imgi.shape
                kpts_pre_i = np.array([(kpt + [0.5, 0.5]) * [img_w, img_h] for kpt in kpts_pre[i]])
                kpts_gt_i = np.array([(kpt + [0.5, 0.5]) * [img_w, img_h] for kpt in kpts_gt[i]])

                imagei_pre = imgi.copy()
                imagei_gt = imgi.copy()
                imagei_pre = paint(imagei_pre, kpts_pre_i)
                imagei_gt = paint(imagei_gt, kpts_gt_i)
                cv2.imwrite(f'./exp/output_114/{index}_{i}_pre.jpg', imagei_pre)
                cv2.imwrite(f'./exp/output_114/{index}_{i}_gt.jpg', imagei_gt)

avg_pre_cost = sum(time_cost_list) / len(time_cost_list)

In [ ]:
avg_pre_cost

测试mAP计算模型

In [ ]:
import torch
import cv2
import numpy as np
import torch
from rlepose.models import builder
from rlepose.utils.config import update_config
from sklearn.metrics import mean_squared_error

cfg = update_config("./configs/256x192_res50_regress-flow_freihand.yaml")
device = torch.device('cuda')
m = builder.build_sppe(cfg.MODEL, preset_cfg=cfg.DATA_PRESET)  # 根据cfg的配置信息构建模型

m.load_state_dict(torch.load("/home/louxd/ldlib/res-loglikelihood-regression-master/weights/model_114.pth", map_location='cpu'), 
                  strict=True)  # 加载权重

m.cuda(device)  # 把模型放到gpu中

with torch.no_grad():
    gt_val_dataset = builder.build_dataset(cfg.DATASET.VAL, preset_cfg=cfg.DATA_PRESET, train=False, heatmap2coord=cfg.TEST.HEATMAP2COORD)
    gt_val_sampler = torch.utils.data.distributed.DistributedSampler(
        gt_val_dataset, num_replicas=1, rank=0)

    gt_val_loader = torch.utils.data.DataLoader(
        gt_val_dataset, batch_size=16, shuffle=False, num_workers=20, drop_last=False, sampler=gt_val_sampler)
    
    
    rmse_list = []
    oks_list = []
    time_cost_list = []

    from rlepose.utils.valid_utils_lxd import compute_RMSE, calculate_oks_pt2, paint, calculate_mAP

    for index, (inps, labels) in enumerate(gt_val_loader):
        inps = inps.cuda(device)
        for k, _ in labels.items():
            if k == 'type':
                continue
            
            labels[k] = labels[k].cuda(device)

        # output = m(inps, labels)
        output = m(inps)

        if isinstance(inps, list):
            batch_size = inps[0].size(0)
        else:
            batch_size = inps.size(0)

        kpts_gt = labels['target_uv'].cpu().numpy().reshape(-1, 21, 2) # size * 42
        kpts_pre = output.pred_jts.cpu().numpy().reshape(-1, 21, 2)

        
        # print & draw
        for i in range(len(inps)):
            #print(f'batch:{batch_idx}_{i}')
            imgi = inps[i].cpu().numpy()
            imgi = np.transpose(imgi, (1, 2, 0))
            imgi = (imgi + np.array([0.480, 0.457, 0.406], dtype=np.float32)) * np.array([255., 255., 255.], dtype=np.float32)
            imgi = cv2.cvtColor(imgi, cv2.COLOR_BGR2RGB) 
            img_h, img_w, _ = imgi.shape
            kpts_pre_i = np.array([(kpt + [0.5, 0.5]) * [img_w, img_h] for kpt in kpts_pre[i]])
            kpts_gt_i = np.array([(kpt + [0.5, 0.5]) * [img_w, img_h] for kpt in kpts_gt[i]])

            # cal mAP and rmse
            # rmsei = compute_RMSE(kpts_pre_i, kpts_gt_i)
            msei = mean_squared_error(kpts_pre_i, kpts_gt_i)
            rmsei = np.sqrt(msei)
            oksi = calculate_oks_pt2(kpts_pre_i, kpts_gt_i)
            rmse_list.append(rmsei)
            oks_list.append(oksi)

            imagei_pre = imgi.copy()
            imagei_gt = imgi.copy()
            imagei_pre = paint(imagei_pre, kpts_pre_i)
            imagei_gt = paint(imagei_gt, kpts_gt_i)
            cv2.imwrite(f'./exp/output/{index}_{i}_pre.jpg', imagei_pre)
            cv2.imwrite(f'./exp/output/{index}_{i}_gt.jpg', imagei_gt)


    rmse = sum(rmse_list) / len(rmse_list)
    mAP_info_str = calculate_mAP(oks_list)

    ap = mAP_info_str['mAP']




In [ ]:
input_size = cfg.DATA_PRESET['IMAGE_SIZE']
output_size = cfg.DATA_PRESET['HEATMAP_SIZE']

In [ ]:
output_size

测试on camera的图像归一化

In [3]:
import torch
import cv2
import numpy as np
import torch
from rlepose.models import builder
from rlepose.utils.config import update_config
from rlepose.utils.valid_utils_lxd import paint
cfg = update_config("./configs/256x192_res50_regress-flow_freihand.yaml")
from rlepose.utils.transforms import im_to_torch

device = torch.device('cuda')
m = builder.build_sppe(cfg.MODEL, preset_cfg=cfg.DATA_PRESET)  # 根据cfg的配置信息构建模型

m.load_state_dict(torch.load("/home/louxd/ldlib/res-loglikelihood-regression-master/weights/model_114.pth", map_location='cpu'), 
                  strict=True)  # 加载权重

m.cuda(device)  # 把模型放到gpu中

input_size = cfg.DATA_PRESET['IMAGE_SIZE']
output_size = cfg.DATA_PRESET['HEATMAP_SIZE']

with torch.no_grad():
    # gt_val_dataset = builder.build_dataset(cfg.DATASET.VAL, preset_cfg=cfg.DATA_PRESET, train=False, heatmap2coord=cfg.TEST.HEATMAP2COORD)
    # gt_val_sampler = torch.utils.data.distributed.DistributedSampler(
    #     gt_val_dataset, num_replicas=1, rank=0)

    # gt_val_loader = torch.utils.data.DataLoader(
    #     gt_val_dataset, batch_size=16, shuffle=False, num_workers=20, drop_last=False, sampler=gt_val_sampler)
    dir = "/home/louxd/dataset/FreiHand/FreiHAND_pub_v2/evaluation/rgb/"
    img_format = ".jpg"

    for j in range(0, 1000):
        img_name = f"{j:08d}{img_format}"
        # 拼接完整的文件路径
        img_path = dir + img_name
        # print(img_path)
        # 读取图片
        init_img = cv2.imread(img_path)

        # init_img = cv2.imread("/home/louxd/dataset/FreiHand/FreiHAND_pub_v2/evaluation/rgb/00000006.jpg")
        src = cv2.cvtColor(init_img, cv2.COLOR_BGR2RGB)
        height, width = src.shape[0], src.shape[1]
        label = {
        # 'bbox': (xmin, ymin, xmax, ymax),
        'bbox': (1, 1, 2, 2), 
        'width': width,
        'height': height,
        'joints_3d': 0
        }

        input_size = cfg.DATA_PRESET['IMAGE_SIZE']
        heatmap_size = cfg.DATA_PRESET['HEATMAP_SIZE']
        
        imgwidth, imght = label['width'], label['height']
        assert imgwidth == src.shape[1] and imght == src.shape[0]

        inp_h, inp_w = input_size
        img = cv2.resize(src, (int(inp_w), int(inp_h)))
        img = im_to_torch(img)
        img[0].add_(-0.406)
        img[1].add_(-0.457)
        img[2].add_(-0.480)

        inps = torch.from_numpy(np.expand_dims(img, axis=0))
        inps = inps.cuda(device)
        output = m(inps)

        kpts_pre = output.pred_jts.cpu().numpy().reshape(-1, 21, 2)

        

        for i in range(len(inps)):
            imgi = inps[i].cpu().numpy()
            imgi = np.transpose(imgi, (1, 2, 0))
            imgi = (imgi + np.array([0.480, 0.457, 0.406], dtype=np.float32))  * np.array([255., 255., 255.], dtype=np.float32)
            imgi = cv2.cvtColor(imgi, cv2.COLOR_BGR2RGB) 
            img_h, img_w, _ = imgi.shape
            imagei_pre = imgi.copy()
            kpts_pre_i = np.array([(kpt + [0.5, 0.5]) * [img_w, img_h] for kpt in kpts_pre[i]])
            imagei_pre = paint(imagei_pre, kpts_pre_i)
            # cv2.imshow("1", imagei_pre)
            # cv2.imshow("0", init_img)
            # cv2.waitKey(0)
            cv2.imwrite(f'./exp/output/{j}.jpg', imagei_pre)
            # cv2.imwrite(f'./exp/out/init.jpg', init_img)
    

    gt_val_dataset = builder.build_dataset(cfg.DATASET.VAL, preset_cfg=cfg.DATA_PRESET, train=False, heatmap2coord=cfg.TEST.HEATMAP2COORD)
    gt_val_sampler = torch.utils.data.distributed.DistributedSampler(
        gt_val_dataset, num_replicas=1, rank=0)

    gt_val_loader = torch.utils.data.DataLoader(
        gt_val_dataset, batch_size=16, shuffle=False, num_workers=20, drop_last=False, sampler=gt_val_sampler)
    
    
    rmse_list = []
    oks_list = []
    time_cost_list = []

    from rlepose.utils.valid_utils_lxd import compute_RMSE, calculate_oks_pt2, paint, calculate_mAP

    for index, (inps, labels) in enumerate(gt_val_loader):
        inps = inps.cuda(device)
        for k, _ in labels.items():
            if k == 'type':
                continue
            
            labels[k] = labels[k].cuda(device)

        # output = m(inps, labels)
        output = m(inps)

        if isinstance(inps, list):
            batch_size = inps[0].size(0)
        else:
            batch_size = inps.size(0)

        kpts_gt = labels['target_uv'].cpu().numpy().reshape(-1, 21, 2) # size * 42
        kpts_pre = output.pred_jts.cpu().numpy().reshape(-1, 21, 2)

        
        # print & draw
        for i in range(len(inps)):
            #print(f'batch:{batch_idx}_{i}')
            imgi = inps[i].cpu().numpy()
            imgi = np.transpose(imgi, (1, 2, 0))
            imgi = (imgi + np.array([0.480, 0.457, 0.406], dtype=np.float32)) * np.array([255., 255., 255.], dtype=np.float32)
            imgi = cv2.cvtColor(imgi, cv2.COLOR_BGR2RGB) 
            img_h, img_w, _ = imgi.shape
            kpts_pre_i = np.array([(kpt + [0.5, 0.5]) * [img_w, img_h] for kpt in kpts_pre[i]])
            kpts_gt_i = np.array([(kpt + [0.5, 0.5]) * [img_w, img_h] for kpt in kpts_gt[i]])


            imagei_pre = imgi.copy()
            imagei_gt = imgi.copy()
            imagei_pre = paint(imagei_pre, kpts_pre_i)
            imagei_gt = paint(imagei_gt, kpts_gt_i)
            cv2.imwrite(f'./exp/output_1/{16 * index + i}_{i}_pre.jpg', imagei_pre)
            # cv2.imwrite(f'./exp/output_1/{index}_{i}_gt.jpg', imagei_gt)
    


/home/louxd/dataset/FreiHand/FreiHAND_pub_v2/evaluation/rgb/00000000.jpg
/home/louxd/dataset/FreiHand/FreiHAND_pub_v2/evaluation/rgb/00000001.jpg
/home/louxd/dataset/FreiHand/FreiHAND_pub_v2/evaluation/rgb/00000002.jpg
/home/louxd/dataset/FreiHand/FreiHAND_pub_v2/evaluation/rgb/00000003.jpg
/home/louxd/dataset/FreiHand/FreiHAND_pub_v2/evaluation/rgb/00000004.jpg
/home/louxd/dataset/FreiHand/FreiHAND_pub_v2/evaluation/rgb/00000005.jpg
/home/louxd/dataset/FreiHand/FreiHAND_pub_v2/evaluation/rgb/00000006.jpg
/home/louxd/dataset/FreiHand/FreiHAND_pub_v2/evaluation/rgb/00000007.jpg
/home/louxd/dataset/FreiHand/FreiHAND_pub_v2/evaluation/rgb/00000008.jpg
/home/louxd/dataset/FreiHand/FreiHAND_pub_v2/evaluation/rgb/00000009.jpg
/home/louxd/dataset/FreiHand/FreiHAND_pub_v2/evaluation/rgb/00000010.jpg
/home/louxd/dataset/FreiHand/FreiHAND_pub_v2/evaluation/rgb/00000011.jpg
/home/louxd/dataset/FreiHand/FreiHAND_pub_v2/evaluation/rgb/00000012.jpg
/home/louxd/dataset/FreiHand/FreiHAND_pub_v2/evalua

In [ ]:
init_img.shape

In [ ]:
import matplotlib.pyplot as plt

thresholds = np.arange(0.5, 1., 0.05)
num_thresholds = len(thresholds)
average_precision = mAP_info_str.values()

plt.plot(thresholds, average_precision, '-o')
plt.xlabel('Threshold')
plt.ylabel('Average Precision')
plt.title('mAP Curve')
plt.grid(True)
plt.show()